In [ ]:
!pip install transformers
!pip install sentencepiece
!pip install tf-models-official

In [2]:
import os
import pandas as pd
from transformers import T5Tokenizer, TFT5ForConditionalGeneration, T5Config
import tensorflow as tf
import numpy as np
from official.nlp import optimization
import datetime
GOOGLE_DRV = '/content/drive'
CHECKPOINT_PATH = GOOGLE_DRV + '/MyDrive/Colab_Notebooks/model'
TRAIN_DATA_PATH = GOOGLE_DRV + '/MyDrive/Colab_Notebooks/data/train.csv'

# Build model architecture
restore model architecture and model weight didn't work because there are custom optimizer in model.
for now build model architecture first then import weights.

In [3]:
T5_MODEL_VER = 0
t5_model_variants = ("t5-small", "t5-base", "t5-large", "t5-3b", "t5-11b")
t5_model = TFT5ForConditionalGeneration.from_pretrained(t5_model_variants[T5_MODEL_VER])
epochs = 20
batch_size = 32
init_lr = 1e-4

steps_per_epoch = 1406
num_train_steps = steps_per_epoch * epochs
num_warmup_steps = num_train_steps // 10


optimizer = optimization.create_optimizer(
    init_lr=init_lr,
    num_train_steps=num_train_steps,
    num_warmup_steps=num_warmup_steps,
    optimizer_type='adamw')
#optimizer = tf.keras.optimizers.Adam(init_lr)

#metrics = [tf.keras.metrics.SparseTopKCategoricalAccuracy(name='accuracy')]

t5_model.compile(optimizer=optimizer, loss='passthrough')


t5_checkpoint = tf.keras.callbacks.ModelCheckpoint(
    filepath=CHECKPOINT_PATH,
    save_weights_only=True,
    monitor='loss',
    mode='min',
    save_best_only=True)

log_dir = 'tensorboard'
start_profile_batch = steps_per_epoch + 10
stop_profile_batch = start_profile_batch + 100
#profile_range = f"{start_profile_batch},{stop_profile_batch}"
log_path = log_dir + "/t5/" + datetime.datetime.now().strftime("%Y-%m-%d_%H:%M:%S")
t5_tensorboard = tf.keras.callbacks.TensorBoard(log_dir=log_path, histogram_freq=1,
                                                #profile_batch=profile_range,
                                                update_freq=20)

t5_earlystop = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=3)
callbacks = [t5_checkpoint, t5_tensorboard, t5_earlystop]
# CHECKPOINT_PATH = 't5_model/DisasterTweets-1'
# checkpoint = tf.train.Checkpoint(t5_model)
# checkpoint.restore(CHECKPOINT_PATH)
t5_model.summary()

All model checkpoint layers were used when initializing TFT5ForConditionalGeneration.

All the layers of TFT5ForConditionalGeneration were initialized from the model checkpoint at t5-small.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFT5ForConditionalGeneration for predictions without further training.


No loss specified in compile() - the model's internal loss computation will be used as the loss. Don't panic - this is a common way to train TensorFlow models in Transformers! To disable this behaviour please pass a loss argument, or explicitly pass `loss=None` if you do not want your model to compute a loss.


Model: "tft5_for_conditional_generation"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 shared (Embedding)          multiple                  16449536  
                                                                 
 encoder (TFT5MainLayer)     multiple                  35330816  
                                                                 
 decoder (TFT5MainLayer)     multiple                  41625344  
                                                                 
Total params: 60,506,624
Trainable params: 60,506,624
Non-trainable params: 0
_________________________________________________________________


# Load Weights into model

In [4]:
t5_model.load_weights(CHECKPOINT_PATH)

# Import data and build tf.dataset

In [5]:
train_df = pd.read_csv(TRAIN_DATA_PATH)
EXAMPLE_SIZE = train_df.shape[0]
train_df = train_df.drop(['id', 'keyword', 'location'], axis=1)
train_df = train_df.rename(columns={"text": "input_ids", "target": "labels"})
train_df['labels'] = train_df.labels.map(lambda x : 'positive' if x == 1 else 'negative')
TASK_PREFIX = 'DisasterTweets: '
train_df['input_ids'] = train_df.input_ids.map(lambda x : TASK_PREFIX + x)

In [6]:
T5_MODEL_VER = 0
t5_model_variants = ("t5-small", "t5-base", "t5-large", "t5-3b", "t5-11b")
t5_tokenizer = T5Tokenizer.from_pretrained(t5_model_variants[T5_MODEL_VER])
in_batch_encoding = t5_tokenizer(train_df['input_ids'].tolist(), padding=True, return_tensors="tf")
targ_batch_encoding = t5_tokenizer(train_df['labels'].astype(str).tolist(),  max_length=2, return_tensors="tf")
ds_train = tf.data.Dataset.from_tensor_slices({'input_ids' : in_batch_encoding.input_ids,
                             'attention_mask' : in_batch_encoding.attention_mask,
                              'labels' : targ_batch_encoding.input_ids,
                              'decoder_attention_mask' : targ_batch_encoding.attention_mask})
# split train/val set
def split_dataset(ds, ds_size, val_split=0.1):
    train_size = int((1 - val_split) * ds_size)
    val_size = int(val_split * ds_size)
    
    train_ds = ds.take(train_size)    
    val_ds = ds.skip(train_size).take(val_size)
    
    return train_ds, val_ds
# prefetch, shuffle and batch the dataset
def dataset_prepare(dataset, batch_size=32, training=True):
    dataset = dataset.cache().prefetch(buffer_size=tf.data.AUTOTUNE)
    if training:
        dataset = dataset.shuffle(1*1024)
    dataset = dataset.batch(batch_size)
    return dataset
dataset = dataset_prepare(ds_train)
ds_train, ds_val = split_dataset(dataset, EXAMPLE_SIZE)

/usr/local/lib/python3.8/dist-packages/transformers/models/t5/tokenization_t5.py:163: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-small automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  warnings.warn(
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (

# Retrain

In [7]:
history = t5_model.fit(
            x=ds_train,
            validation_data=ds_val,
            callbacks=callbacks,
            epochs=epochs)

Epoch 1/20


Instructions for updating:
Lambda fuctions will be no more assumed to be used in the statement where they are used, or at least in the same block. https://github.com/tensorflow/tensorflow/issues/56089


238/238 [==============================] - 114s 370ms/step - loss: 0.0732
Epoch 2/20
238/238 [==============================] - 88s 369ms/step - loss: 0.0683
Epoch 3/20
238/238 [==============================] - 85s 359ms/step - loss: 0.0605
Epoch 4/20
238/238 [==============================] - 84s 355ms/step - loss: 0.0553
Epoch 5/20
238/238 [==============================] - 87s 366ms/step - loss: 0.0532
Epoch 6/20
238/238 [==============================] - 85s 357ms/step - loss: 0.0495
Epoch 7/20
238/238 [==============================] - 85s 359ms/step - loss: 0.0434
Epoch 8/20
238/238 [==============================] - 82s 346ms/step - loss: 0.0442
Epoch 9/20
238/238 [==============================] - 89s 373ms/step - loss: 0.0380
Epoch 10/20
238/238 [==============================] - 85s 358ms/step - loss: 0.0374
Epoch 11/20
238/238 [==============================] - 85s 358ms/step - loss: 0.0372
Epoch 12/20
238/238 [==============================] - 87s 365ms/step - loss: 0.0336